In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import pprint
import project_helper


from tqdm import tqdm

In [2]:
df = pd.read_csv("sp500_w_addl_id_with_cik.csv")

C:\Users\jacky\AppData\Local\Temp\ipykernel_60384\2057541701.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("sp500_w_addl_id_with_cik.csv")


In [3]:
df = df.dropna()

In [4]:
#formatting to be in accord with sec edgar links' formats, i.e. 10-digit CIK
df.loc[:, 'cik'] = df['cik'].dropna().astype(int).astype(str).str.zfill(10)

In [5]:
df.loc[:, 'date'] = pd.to_datetime(df['date'])

In [6]:
df

,Unnamed: 0,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret,cik,sic,naics,gind,gsubind
0,0,2011-01-31,60986.0,NEWELL RUBBERMAID INC,65122910,11.0,1.0,3089.0,NWL,7875,01,1989-04-27,2022-03-31,0.058856,0000814453,3990,3399,252010,25201050
1,1,2011-01-31,85914.0,BEST BUY COMPANY INC,08651610,11.0,1.0,5731.0,BBY,2184,01,1999-06-30,2022-03-31,-0.008457,0000764478,5731,443142,255040,25504020
2,2,2011-01-31,80711.0,APARTMENT INVESTMENT & MGMT CO,03748R10,18.0,1.0,6798.0,AIV,30490,01,2003-03-14,2020-12-18,-0.010836,0000922864,6798,531110,601010,60101060
3,3,2011-01-31,59176.0,AMERICAN EXPRESS CO,02581610,11.0,1.0,6141.0,AXP,1447,01,1976-07-01,2022-03-31,0.014912,0000004962,6141,522210,402020,40202010
4,4,2011-01-31,75100.0,TIFFANY & CO NEW,88654710,11.0,1.0,5944.0,TIF,13646,01,2000-06-21,2021-01-06,-0.066485,0000098246,5944,448310,255040,25504040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66113,66113,2021-12-31,83143.0,IRON MOUNTAIN INC NEW,46284V10,18.0,1.0,4226.0,IRM,62374,1,2009-01-06,2022-03-31,0.165240,0001020569,6798,531120,601010,60101080
66114,66114,2021-12-31,76605.0,AUTOZONE INC,05333210,11.0,1.0,5531.0,AZO,23809,1,1997-01-02,2022-03-31,0.153720,0000866787,5531,441310,255040,25504050
66115,66115,2021-12-31,60097.0,MEDTRONIC PLC,G5960L10,12.0,1.0,3845.0,MDT,7228,1,1986-10-23,2022-03-31,-0.024555,0001613103,3845,334510,351010,35101010
66116,66116,2021-12-31,27828.0,H P INC,40434L10,11.0,1.0,3571.0,HPQ,5606,1,1974-10-17,2022-03-31,0.074830,0000047217,3570,33411,452020,45202030


In [7]:
months = df['date'].unique()
#filter out months
months = [m for m in months if m>=np.datetime64('2016-01-01')]

In [8]:
month_cik = dict()
for m in months:
    ticker_cik_list = df[df['date'] == m][['ticker', 'cik']].to_numpy()
    month_cik[m] = ticker_cik_list

In [9]:
month_cik[months[0]]

array([['NBL', '0000072207'],
       ['PLD', '0001045609'],
       ['MOS', '0001285785'],
       ['URI', '0001067701'],
       ['NVDA', '0001045810'],
       ['OKE', '0001039684'],
       ['ZION', '0000109380'],
       ['APH', '0000820313'],
       ['GE', '0000040545'],
       ['SYK', '0000310764'],
       ['TGNA', '0000039899'],
       ['TSN', '0000100493'],
       ['ALXN', '0000899866'],
       ['DIS', '0001744489'],
       ['MCO', '0001059556'],
       ['MKC', '0000063754'],
       ['HP', '0000046765'],
       ['STX', '0001137789'],
       ['HRS', '0000202058'],
       ['JEC', '0000052988'],
       ['FDX', '0001048911'],
       ['VTR', '0000740260'],
       ['MTB', '0000036270'],
       ['EW', '0001099800'],
       ['MAT', '0000063276'],
       ['PCP', '0000079958'],
       ['DISCA', '0001437107'],
       ['BMY', '0000014272'],
       ['MU', '0000723125'],
       ['CMI', '0000026172'],
       ['IRM', '0001020569'],
       ['SBUX', '0000829224'],
       ['RTN', '0001047122'],
       

In [10]:
#insert a starting point for the months array for range exclusion to be used later
months.insert(0, np.datetime64('2016-01-01T00:00:00.000000000'))

In [11]:
# for m in months:
#     print(m)

me = months[1]

def generate_cik(me):
    print("Generating CIKs")
    ticker_cik = month_cik[me]
    cik_lookup = dict()
    for i in ticker_cik:
        cik_lookup[i[0]] = i[1]
    return cik_lookup

In [12]:
sec_api = project_helper.SecAPI()

from bs4 import BeautifulSoup
import requests

HEADER = {'Host': 'www.sec.gov', 
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

# from selenium import webdriver
# import time
# import os

# PROJECT_ROOT = os.path.abspath(os.path.dirname('/Users/jacky/Desktop/NLP_Project/'))
# DRIVER_BIN = os.path.join(PROJECT_ROOT, "chromedriver")

def get_sec_data(cik, doc_type, start=0, count=60):
    rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
        '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
        .format(cik, doc_type, start, count)
#     browser=webdriver.Chrome(executable_path = DRIVER_BIN)

#     # get source code
#     browser.get(rss_url)
#     sec_data = browser.page_source
#     time.sleep(2)

#     # close web browser
#     browser.close()
    sec_data = requests.get(rss_url, headers=HEADER)
    feed = BeautifulSoup(sec_data.content, 'html')
    entries = [
        (
            entry.content.find('filing-href').getText(),
            entry.content.find('filing-type').getText(),
            entry.content.find('filing-date').getText())
        for entry in feed.find_all('entry')]

    return entries

In [13]:
def download(ms, me): #m for month, start and ending month of the period of S&P 500 considered
    print("Downloading for", me)
    sec_data = {}
    
    cik_lookup = generate_cik(me)
    b = 1
    for ticker, cik in cik_lookup.items():
        print(b, "Looking Up Ticker:", ticker)
        sec_data[ticker] = get_sec_data(cik, '10')
        b += 1
    raw_fillings_by_ticker = {}

    me_str = str(pd.to_datetime(me).date())
    ms_str = str(pd.to_datetime(ms).date())
    
    c = 1
    for ticker, data in sec_data.items():
        print(c, "Pulling Ticker:", ticker)
        raw_fillings_by_ticker[ticker] = {}
        for index_url, file_type, file_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
            if (file_type == '10-K' or file_type == '10-Q') and (file_date > ms_str) and (file_date < me_str):
                file_url = index_url.replace('-index.htm', '.txt').replace('.txtl', '.txt')            

                fil = requests.get(file_url, headers=HEADER)
                raw_fillings_by_ticker[ticker][file_date] = fil.content.decode('utf-8')
        c += 1
    return raw_fillings_by_ticker

In [14]:
import re
def get_documents(text):
    extracted_docs = []
    
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')   
    
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(text)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(text)]
    
    for doc_start_i, doc_end_i in zip(doc_start_is, doc_end_is):
            extracted_docs.append(text[doc_start_i:doc_end_i])
    
    return extracted_docs

def filling(ms, me):
    raw_fillings_by_ticker = download(ms, me)
    print("Getting raw fillings")
    filling_documents_by_ticker = {}
    c = 1
    for ticker, raw_fillings in raw_fillings_by_ticker.items():
        print(c)
        filling_documents_by_ticker[ticker] = {}
        for file_date, filling in tqdm(raw_fillings.items(), desc='Getting Documents from {} Fillings'.format(ticker), unit='filling'):
            filling_documents_by_ticker[ticker][file_date] = get_documents(filling)
        c += 1
    return filling_documents_by_ticker

In [15]:
def get_document_type(doc):
    
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    
    doc_type = type_pattern.findall(doc)[0][len('<TYPE>'):] 
    
    return doc_type.lower()

In [16]:
import lxml
import cchardet

def remove_html_tags(text):
    text = BeautifulSoup(text, 'lxml').get_text()
    
    return text

def clean_text(text):
    text = text.lower()
    text = remove_html_tags(text)
    
    return text

In [18]:

def save_file(ms, me):
    filling_documents_by_ticker = filling(ms, me)
    cik_lookup = generate_cik(me)
    print("Saving files to txt")
    ten_ks_by_ticker = {}
    for ticker, filling_documents in filling_documents_by_ticker.items():
        ten_ks_by_ticker[ticker] = []
        for file_date, documents in filling_documents.items():
            for document in documents:
                if get_document_type(document) == '10-k' or get_document_type(document) == '10-q':
                    file_name = ticker + '-' + cik_lookup[ticker] + '-' + file_date + '.txt'
                    print("Downloading", file_name)
                    with open('data_4860/' + file_name, 'w', encoding='utf-8') as f:
                        doc = clean_text(document)
                        f.write(doc)

In [20]:
for i in range(48, 60):
    print("Month:", i+1)
    if i == 1:
        continue
#     if i < 3: add this if statement if you just want to test on first couple of months
    ms = months[i]
    me = months[i+1]
    save_file(ms, me)

Month: 49
Generating CIKs
1 Looking Up Ticker: REGN
2 Looking Up Ticker: GPS
3 Looking Up Ticker: UDR
4 Looking Up Ticker: NOW
5 Looking Up Ticker: CFG
6 Looking Up Ticker: BKR
7 Looking Up Ticker: AWK
8 Looking Up Ticker: CVS
9 Looking Up Ticker: CHRW
10 Looking Up Ticker: GIS
11 Looking Up Ticker: PH
12 Looking Up Ticker: VRSN
13 Looking Up Ticker: NEM
14 Looking Up Ticker: HPQ
15 Looking Up Ticker: MRO
16 Looking Up Ticker: SBUX
17 Looking Up Ticker: HRB
18 Looking Up Ticker: INTC
19 Looking Up Ticker: LLY
20 Looking Up Ticker: NLSN
21 Looking Up Ticker: YUM
22 Looking Up Ticker: OMC
23 Looking Up Ticker: LHX
24 Looking Up Ticker: SJM
25 Looking Up Ticker: MSCI
26 Looking Up Ticker: LNT
27 Looking Up Ticker: SYK
28 Looking Up Ticker: BSX
29 Looking Up Ticker: BAC
30 Looking Up Ticker: CINF
31 Looking Up Ticker: CBRE
32 Looking Up Ticker: TSN
33 Looking Up Ticker: HUM
34 Looking Up Ticker: VLO
35 Looking Up Ticker: SCHW
36 Looking Up Ticker: SYF
37 Looking Up Ticker: CBOE
38 Looking 

304 Looking Up Ticker: VAR
305 Looking Up Ticker: CCI
306 Looking Up Ticker: APA
307 Looking Up Ticker: EOG
308 Looking Up Ticker: CTXS
309 Looking Up Ticker: TMO
310 Looking Up Ticker: DLTR
311 Looking Up Ticker: AVGO
312 Looking Up Ticker: NVR
313 Looking Up Ticker: XRX
314 Looking Up Ticker: FBHS
315 Looking Up Ticker: NUE
316 Looking Up Ticker: ODFL
317 Looking Up Ticker: COP
318 Looking Up Ticker: CVX
319 Looking Up Ticker: EMR
320 Looking Up Ticker: EXPE
321 Looking Up Ticker: EXPD
322 Looking Up Ticker: WAT
323 Looking Up Ticker: BIIB
324 Looking Up Ticker: FISV
325 Looking Up Ticker: ILMN
326 Looking Up Ticker: CSX
327 Looking Up Ticker: ROST
328 Looking Up Ticker: EBAY
329 Looking Up Ticker: LW
330 Looking Up Ticker: NI
331 Looking Up Ticker: CMCSA
332 Looking Up Ticker: FTV
333 Looking Up Ticker: BA
334 Looking Up Ticker: AIG
335 Looking Up Ticker: EL
336 Looking Up Ticker: MCK
337 Looking Up Ticker: EA
338 Looking Up Ticker: INCY
339 Looking Up Ticker: PBCT
340 Looking Up Ti

2 Pulling Ticker: GPS


3 Pulling Ticker: UDR


4 Pulling Ticker: NOW


...

148 Pulling Ticker: FCX


149 Pulling Ticker: NSC


150 Pulling Ticker: NWSA


151 Pulling Ticker: BMY


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [18]:
i = 1
ms = months[i]
me = months[i+1]
save_file(ms, me)

Generating CIKs
1 Looking Up Ticker: TJX


C:\Users\jacky\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


2 Looking Up Ticker: CTL
3 Looking Up Ticker: NFLX
4 Looking Up Ticker: GME
5 Looking Up Ticker: APH
6 Looking Up Ticker: AVY
7 Looking Up Ticker: PCAR
8 Looking Up Ticker: RL
9 Looking Up Ticker: LMT
10 Looking Up Ticker: AMZN
11 Looking Up Ticker: WRK
12 Looking Up Ticker: PCG
13 Looking Up Ticker: PEG
14 Looking Up Ticker: HCA
15 Looking Up Ticker: QRVO
16 Looking Up Ticker: PNC
17 Looking Up Ticker: AAP
18 Looking Up Ticker: PRU
19 Looking Up Ticker: SNA
20 Looking Up Ticker: VRTX
21 Looking Up Ticker: ADP
22 Looking Up Ticker: GE
23 Looking Up Ticker: BXP
24 Looking Up Ticker: CSCO
25 Looking Up Ticker: RRC
26 Looking Up Ticker: THC
27 Looking Up Ticker: K
28 Looking Up Ticker: TXN
29 Looking Up Ticker: COST
30 Looking Up Ticker: BIIB
31 Looking Up Ticker: MSI
32 Looking Up Ticker: EW
33 Looking Up Ticker: EA
34 Looking Up Ticker: EFX
35 Looking Up Ticker: VAR
36 Looking Up Ticker: GT
37 Looking Up Ticker: XRX
38 Looking Up Ticker: GAS
39 Looking Up Ticker: VRSK
40 Looking Up Tick

310 Looking Up Ticker: FMC
311 Looking Up Ticker: ESS
312 Looking Up Ticker: EXPE
313 Looking Up Ticker: SEE
314 Looking Up Ticker: FIS
315 Looking Up Ticker: BLK
316 Looking Up Ticker: AIV
317 Looking Up Ticker: VZ
318 Looking Up Ticker: DISCA
319 Looking Up Ticker: MAT
320 Looking Up Ticker: FRT
321 Looking Up Ticker: O
322 Looking Up Ticker: COF
323 Looking Up Ticker: SLB
324 Looking Up Ticker: BBY
325 Looking Up Ticker: MCK
326 Looking Up Ticker: LM
327 Looking Up Ticker: BDX
328 Looking Up Ticker: FAST
329 Looking Up Ticker: BCR
330 Looking Up Ticker: PSX
331 Looking Up Ticker: MNK
332 Looking Up Ticker: ESV
333 Looking Up Ticker: NDAQ
334 Looking Up Ticker: KEY
335 Looking Up Ticker: POM
336 Looking Up Ticker: FTI
337 Looking Up Ticker: MKC
338 Looking Up Ticker: PBI
339 Looking Up Ticker: AN
340 Looking Up Ticker: LUV
341 Looking Up Ticker: WY
342 Looking Up Ticker: DLTR
343 Looking Up Ticker: MAR
344 Looking Up Ticker: DE
345 Looking Up Ticker: DTE
346 Looking Up Ticker: ABC
34

2 Pulling Ticker: CTL


...

498


Getting Documents from ORCL Fillings: 0filling [00:00, ?filling/s]


Generating CIKs
Saving files to txt
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Wri

Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleani

Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleaning
Writing
Cleani

In [19]:
len(months)

73